# STEP 1 : Load all the necessary libraries

In [32]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import os 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models,layers,optimizers,utils

# STEP 2 : Data Preparation

In [33]:
dataframe=pd.read_csv("Concrete_Data_Yeh.csv")
dataframe.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [34]:
Concrete= dataframe.sample(frac=1)
Concrete.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
119,425.0,106.3,0.0,151.4,18.6,936.0,803.7,28,61.80
691,212.0,141.3,0.0,203.5,0.0,973.4,750.0,3,6.81
786,331.0,0.0,0.0,192.0,0.0,978.0,825.0,28,31.45
626,236.0,0.0,0.0,193.0,0.0,968.0,885.0,7,9.99
498,355.0,19.0,97.0,145.0,12.3,967.0,871.0,28,55.45


In [35]:
concrete=Concrete.values
x_train = concrete[:824,:8].astype('float')
train_labels = concrete[:824,8]
y_test = concrete[824:,:8].astype('float')
test_labels = concrete[824:,8]

In [36]:
print(x_train.shape)
print(train_labels.shape)
print(y_test.shape)
print(test_labels.shape)

(824, 8)
(824,)
(206, 8)
(206,)


# Normalization

In [37]:
mean=x_train.mean(axis=0)
x_train-=mean
std=x_train.std(axis=0)
x_train/=std
y_test-=mean
y_test/=std

# Step 3:Slicing for Validation

In [47]:
x_val=x_train[:103] #totl 100% hain jisme se validation me 10% ja rahe hain
partial_x_train=x_train[103:] #or baki partial me ja rahe ha
val_label=train_labels[:103]
partial_train_label=train_labels[103:]

# Step 4:Building Model Network

In [48]:
model=models.Sequential()
model.add(layers.Dense(64,activation='relu',input_shape=(partial_x_train.shape[1],)))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(1)) #regression problem me last me koi bh func nai lagate
model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])

In [49]:
history=model.fit(partial_x_train,partial_train_label,epochs=100,batch_size=30,validation_data=(x_val,val_label))


Train on 721 samples, validate on 103 samples
Epoch 1/100
721/721 [==============================] - 0s 604us/sample - loss: 1399.2087 - mae: 33.4658 - val_loss: 1289.3557 - val_mae: 32.4450
Epoch 2/100
721/721 [==============================] - 0s 74us/sample - loss: 1137.3869 - mae: 29.6128 - val_loss: 988.3866 - val_mae: 27.7822
Epoch 3/100
721/721 [==============================] - 0s 56us/sample - loss: 827.3642 - mae: 24.4455 - val_loss: 658.2984 - val_mae: 21.7277
Epoch 4/100
721/721 [==============================] - 0s 60us/sample - loss: 526.5835 - mae: 18.5122 - val_loss: 401.4778 - val_mae: 16.0025
Epoch 5/100
721/721 [==============================] - 0s 58us/sample - loss: 318.1560 - mae: 14.2674 - val_loss: 235.0124 - val_mae: 12.3810
Epoch 6/100
721/721 [==============================] - 0s 61us/sample - loss: 225.8628 - mae: 12.1363 - val_loss: 200.1163 - val_mae: 11.3433
Epoch 7/100
721/721 [==============================] - 0s 58us/sample - loss: 204.1576 - mae: 11.5

Epoch 59/100
721/721 [==============================] - 0s 68us/sample - loss: 75.3166 - mae: 6.8191 - val_loss: 96.1422 - val_mae: 7.7891
Epoch 60/100
721/721 [==============================] - 0s 72us/sample - loss: 74.1744 - mae: 6.7366 - val_loss: 93.3741 - val_mae: 7.5809
Epoch 61/100
721/721 [==============================] - 0s 63us/sample - loss: 72.7605 - mae: 6.6738 - val_loss: 95.8844 - val_mae: 7.5082
Epoch 62/100
721/721 [==============================] - 0s 58us/sample - loss: 71.4333 - mae: 6.6122 - val_loss: 91.7592 - val_mae: 7.4620
Epoch 63/100
721/721 [==============================] - 0s 58us/sample - loss: 69.4105 - mae: 6.4851 - val_loss: 91.4062 - val_mae: 7.5087
Epoch 64/100
721/721 [==============================] - 0s 61us/sample - loss: 68.6120 - mae: 6.4519 - val_loss: 90.8483 - val_mae: 7.5124
Epoch 65/100
721/721 [==============================] - 0s 58us/sample - loss: 67.8135 - mae: 6.4018 - val_loss: 90.6205 - val_mae: 7.3582
Epoch 66/100
721/721 [=====

In [50]:
val_mse, val_mae = model.evaluate(x_val, val_label, verbose=0)

In [51]:
model.fit(x_train, train_labels,
          epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(y_test, test_labels)

206/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [52]:
test_mae_score

4.2765274